Load Julia packages (libraries) needed  for the snippets in chapter 0

In [1]:
using StanModels

CmdStan uses a tmp directory to store the output of cmdstan

In [2]:
ProjDir = rel_path_s("..", "scripts", "10")
cd(ProjDir)

### snippet 10.4

In [3]:
d = CSV.read(rel_path("..", "data", "chimpanzees.csv"), delim=';');
df = convert(DataFrame, d);

first(df, 5)

5×8 DataFrame. Omitted printing of 2 columns
│ Row │ actor  │ recipient │ condition │ block  │ trial  │ prosoc_left │
│     │ Int64⍰ │ String⍰   │ Int64⍰    │ Int64⍰ │ Int64⍰ │ Int64⍰      │
├─────┼────────┼───────────┼───────────┼────────┼────────┼─────────────┤
│ 1   │ 1      │ NA        │ 0         │ 1      │ 2      │ 0           │
│ 2   │ 1      │ NA        │ 0         │ 1      │ 4      │ 0           │
│ 3   │ 1      │ NA        │ 0         │ 1      │ 6      │ 1           │
│ 4   │ 1      │ NA        │ 0         │ 1      │ 8      │ 0           │
│ 5   │ 1      │ NA        │ 0         │ 1      │ 10     │ 1           │

Define the Stan language model

In [4]:
m_10_02 = "
data{
    int N;
    int pulled_left[N];
    int prosoc_left[N];
}
parameters{
    real a;
    real bp;
}
model{
    vector[N] p;
    bp ~ normal( 0 , 10 );
    a ~ normal( 0 , 10 );
    for ( i in 1:N ) {
        p[i] = a + bp * prosoc_left[i];
        p[i] = inv_logit(p[i]);
    }
    pulled_left ~ binomial( 1 , p );
}
";

Define the Stanmodel and set the output format to :mcmcchains.

In [5]:
stanmodel = Stanmodel(name="m_10_02",
monitors = ["a", "bp"],
model=m_10_02, output_format=:mcmcchains);


File /Users/rob/.julia/dev/StanModels/scripts/10/tmp/m_10_02.stan will be updated.



Input data for cmdstan

In [6]:
m_10_02_data = Dict("N" => size(df, 1),
"pulled_left" => df[:pulled_left], "prosoc_left" => df[:prosoc_left]);

Sample using cmdstan

In [7]:
rc, chn, cnames = stan(stanmodel, m_10_02_data, ProjDir, diagnostics=false,
  summary=false, CmdStanDir=CMDSTAN_HOME);
# Result rethinking
# Result rethinking
rethinking = "
   mean   sd  5.5% 94.5% n_eff Rhat
a  0.04 0.12 -0.16  0.21   180 1.00
bp 0.57 0.19  0.30  0.87   183 1.01
";

Describe the draws

In [8]:
describe(chn)

Iterations        = 1:1000
Thinning interval = 1
Chains            = 1, 2, 3, 4
Samples per chain = 1000
parameters        = a, bp

Empirical Posterior Estimates
───────────────────────────────────────
parameters
    Mean    SD   Naive SE  MCSE   ESS
 a 0.0473 0.1236   0.0020 0.0034 1000
bp 0.5624 0.1791   0.0028 0.0049 1000

Quantiles
───────────────────────────────────────
parameters
     2.5%   25.0%   50.0%  75.0%  97.5%
 a -0.1923 -0.0363 0.0478 0.1299 0.2943
bp  0.2137  0.4459 0.5632 0.6799 0.9161



End of `10/m10.02s.jl`

*This notebook was generated using [Literate.jl](https://github.com/fredrikekre/Literate.jl).*